In [ ]:
pip install transformers datasets accelerate


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv("/content/drive/MyDrive/model-nlp/dataset-perbaikan.csv")
data.head()


,english,indonesia
0,Agastya,Agastya
1,In Hindu mythology,Dalam mitologi Hindu
2,Agastya is known as a rishi (great sage) who p...,Agastya dikenal sebagai seorang rsi (pendeta b...
3,particularly in the southern regions of India,terutama di wilayah selatan India
4,He is depicted as an elderly man with a long b...,Ia digambarkan sebagai sosok pria tua berjengg...


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-en-id"

# Load tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
from torch.utils.data import Dataset

class TranslationDataset(Dataset):
    def __init__(self, data, tokenizer, source_col, target_col, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = self.data.iloc[idx][self.source_col]
        target_text = self.data.iloc[idx][self.target_col]

        source_encodings = self.tokenizer(
            source_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        target_encodings = self.tokenizer(
            target_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = target_encodings["input_ids"]
        labels[labels == tokenizer.pad_token_id] = -100  # Mask padding token

        return {
            "input_ids": source_encodings["input_ids"].squeeze(),
            "attention_mask": source_encodings["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
        }

# Buat dataset
train_dataset = TranslationDataset(data, tokenizer, "english", "indonesia")


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split

# Split dataset into training and evaluation
train_data, eval_data = train_test_split(data, test_size=0.1)  # 10% untuk evaluasi

# Buat dataset untuk evaluasi
eval_dataset = TranslationDataset(eval_data, tokenizer, "english", "indonesia")

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,  # Jika menggunakan GPU dengan dukungan mixed precision
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Tambahkan eval_dataset di sini
    tokenizer=tokenizer
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-fcf7ef3b2ac6>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aditwinaya123 (aditwinaya123-universitas-amikom-yogyakarta). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,6.238900,5.589191
20,5.437300,4.956817
30,5.083700,4.472161
40,4.665400,4.166363
50,4.304700,3.908502
60,3.882900,3.699943
70,4.095600,3.526600
80,3.676700,3.361259
90,3.627300,3.213951
100,3.423800,3.121089


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[54795]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=175, training_loss=3.869121355329241, metrics={'train_runtime': 2286.7356, 'train_samples_per_second': 0.599, 'train_steps_per_second': 0.077, 'total_flos': 46440759951360.0, 'train_loss': 3.869121355329241, 'epoch': 5.0})

In [ ]:
# Tentukan folder penyimpanan di Google Drive

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Muat model dan tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

# Tentukan folder penyimpanan di Google Drive
output_dir = "/content/drive/MyDrive/model-nlp"

# Simpan model dan tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model dan tokenizer berhasil disimpan di {output_dir}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Model dan tokenizer berhasil disimpan di /content/drive/MyDrive/model-nlp


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model dan tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

# Input teks
input_text = "my father is superman"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate terjemahan
outputs = model.generate(**inputs)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated_text)


Ayahku adalah superman.


In [ ]:
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf

def convert_transformer_to_tflite(model_path, tokenizer_path):
    # Load model dan tokenizer
    model = TFAutoModel.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")
    tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

    # Contoh input untuk konversi
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)

    # Buat model Keras
    outputs = model(input_ids, attention_mask=attention_mask)
    keras_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

    # Converter TFLite
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops
        tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops
    ]

    # Konversi dan simpan
    tflite_model = converter.convert()
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

In [ ]:
from transformers import TFMarianMTModel, MarianTokenizer
import tensorflow as tf
import numpy as np
import os

def convert_marian_to_tflite(model_path, tokenizer_path):
    # Load model dan tokenizer
    model = TFMarianMTModel.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")
    tokenizer = MarianTokenizer.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

    # Tentukan path di Google Drive
    output_dir = "/content/drive/MyDrive/model-nlp/tflite_model"
    os.makedirs(output_dir, exist_ok=True)

    # Buat model prediksi khusus
    class MarianTFLiteModel(tf.keras.Model):
        def __init__(self, model):
            super().__init__()
            self.model = model

        def call(self, inputs):
            input_ids, attention_mask = inputs

            # Generate decoder_input_ids
            decoder_input_ids = tf.fill(
                tf.shape(input_ids),
                tokenizer.pad_token_id
            )

            # Panggil model dengan parameter yang tepat
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids
            )

            # Kembalikan logits atau last_hidden_state
            return outputs.logits

    # Buat model wrapper
    tflite_model = MarianTFLiteModel(model)

    # Input specification
    input_shape = (None, None)
    input_specs = [
        tf.TensorSpec(shape=input_shape, dtype=tf.int32),
        tf.TensorSpec(shape=input_shape, dtype=tf.int32)
    ]

    # Trace model
    concrete_func = tf.function(tflite_model).get_concrete_function(input_specs)

    # Converter TFLite
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]

    # Tambahkan optimasi
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    try:
        # Konversi dan simpan
        tflite_model = converter.convert()
        output_path = os.path.join(output_dir, 'model.tflite')

        with open(output_path, 'wb') as f:
            f.write(tflite_model)

        print(f"Model TFLite berhasil disimpan di: {output_path}")

    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

# Gunakan fungsi
convert_marian_to_tflite(
    "/content/drive/MyDrive/model-nlp/model-translation",
    "/content/drive/MyDrive/model-nlp/model-translation"
)

All PyTorch model weights were used when initializing TFMarianMTModel.

Some weights or buffers of the TF 2.0 model TFMarianMTModel were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Model TFLite berhasil disimpan di: /content/drive/MyDrive/model-nlp/tflite_model/model.tflite


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.5 MB/s eta 0:00:00


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Path ke folder model hasil fine-tuning
fine_tuned_model_path = "/content/drive/MyDrive/model-nlp/model-translation"  # Ubah sesuai lokasi folder

# Load model dan tokenizer dari folder fine-tuning
model = TFAutoModelForSeq2SeqLM.from_pretrained(fine_tuned_model_path, from_pt=True)  # from_pt=True untuk konversi PyTorch ke TensorFlow
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Simpan model dalam format TensorFlow
model.save_pretrained("/content/drive/MyDrive/model-nlp/fine_tuned_model_tf")
tokenizer.save_pretrained("/content/drive/MyDrive/model-nlp/fine_tuned_model_tf")

All PyTorch model weights were used when initializing TFMarianMTModel.

Some weights or buffers of the TF 2.0 model TFMarianMTModel were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


('/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/tokenizer_config.json',
 '/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/special_tokens_map.json',
 '/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/vocab.json',
 '/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/source.spm',
 '/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/target.spm',
 '/content/drive/MyDrive/model-nlp/fine_tuned_model_tf/added_tokens.json')

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Load the Hugging Face model and tokenizer
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

# Save the model in TensorFlow's SavedModel format
saved_model_path = "/content/drive/MyDrive/model-nlp/fine_tuned_model_tf"
tf.saved_model.save(model, saved_model_path)

All PyTorch model weights were used when initializing TFMarianMTModel.

Some weights or buffers of the TF 2.0 model TFMarianMTModel were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)

# Aktifkan debugging
converter.experimental_new_converter = False  # Gunakan konverter lama jika diperlukan
converter.allow_custom_ops = True  # Izinkan operasi kustom

try:
    tflite_model = converter.convert()
    # Simpan model TFLite
    tflite_model_path = "/content/drive/MyDrive/model-nlp/fine_tuned_model.tflite"
    with open(tflite_model_path, "wb") as f:
        f.write(tflite_model)
    print(f"Model berhasil dikonversi ke format .tflite di {tflite_model_path}")
except Exception as e:
    print("Error during conversion:", e)

ValueError: File format not supported: filepath=/content/drive/MyDrive/model-nlp/fine_tuned_model_tf. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/drive/MyDrive/model-nlp/fine_tuned_model_tf, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
# Path ke model TensorFlow yang sudah disimpan
tf_model_path = "/content/drive/MyDrive/model-nlp/fine_tuned_model_tf"

# Load model TensorFlow
model = tf.saved_model.load(tf_model_path)

# Konversi model ke TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
tflite_model = converter.convert()

# Simpan model TFLite
tflite_model_path = "/content/drive/MyDrive/model-nlp/fine_tuned_model.tflite"  # Ubah sesuai lokasi yang diinginkan
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"Model berhasil dikonversi ke format .tflite di {tflite_model_path}")

ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["StridedSlice:", "tf_marian_mt_model_8/model/decoder/strided_slice_4@__inference__wrapped_model_64322"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_83330"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): 'tf.StridedSlice' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["StridedSlice:", "tf_marian_mt_model_8/model/decoder/strided_slice_4@__inference__wrapped_model_64322"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_83330"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall"])): Error code: ERROR_NEEDS_FLEX_OPS
<unknown>:0: error: loc(callsite(callsite(fused["StridedSlice:", "tf_marian_mt_model_8/model/decoder/strided_slice_7@

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
# Load model TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Siapkan input teks
input_text = "He is depicted as an elderly man with a long beard"
inputs = tokenizer(input_text, return_tensors="tf")["input_ids"].numpy()

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], inputs)

# Jalankan inferensi
interpreter.invoke()

# Ambil output tensor
outputs = interpreter.get_tensor(output_details[0]['index'])
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Hasil terjemahan: {translated_text}")

In [ ]:
tflite_model_path = "/content/drive/MyDrive/model-nlp/fine_tuned_model.tflite"  # Ubah sesuai lokasi yang diinginkan
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"Model berhasil dikonversi ke format .tflite di {tflite_model_path}")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
import numpy as np

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TFAutoModelForSeq2SeqLM

# Load model dan tokenizer (dari Hugging Face)
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

# Konversi model dari PyTorch ke TensorFlow
model_tf = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation")

# Simpan model TensorFlow
model_tf.save_pretrained("/content/drive/MyDrive/model-nlp/model-translation/fine_tuned_model_tf")


All PyTorch model weights were used when initializing TFMarianMTModel.

Some weights or buffers of the TF 2.0 model TFMarianMTModel were not initialized from the PyTorch model and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM

# Muat model TensorFlow yang telah disimpan
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model-nlp/model-translation/fine_tuned_model_tf")

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Lakukan konversi
tflite_model = converter.convert()

# Simpan model .tflite
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model berhasil dikonversi ke format .tflite")


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at /content/drive/MyDrive/model-nlp/model-translation/fine_tuned_model_tf.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
import tensorflow as tf

# Muat model TensorFlow
model = tf.saved_model.load("/content/drive/MyDrive/model-nlp/model-translation/tf_model.h5")

# Konversi ke format TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("/content/drive/MyDrive/model-nlp/model-translation/fine_tuned_model_tf")
tflite_model = converter.convert()

# Simpan model dalam format TFLite
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model berhasil disimpan dalam format .tflite")


OSError: SavedModel file does not exist at: /content/drive/MyDrive/model-nlp/model-translation/tf_model.h5/variables/{saved_model.pbtxt|saved_model.pb}